<a href="https://colab.research.google.com/github/Moussaoui-Ghiles/tensorflow-2.0/blob/main/tf_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sklearn

import pandas as pd 
import tensorflow as tf 

import numpy as np 


In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on


train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe



train_y = train.pop("Species")
test_y = test.pop("Species")




def input_fn(features, labels, training = True , batch_size = 256):

  # convert the inputs to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # shuffle and repeat if you are in training mode 

  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)



# feature columns 
#Feature columns describe how to use the input 
my_feature_columns= []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))


# Build a DNN with 2 hidden layers with 30 and 10 hiden nodes each 
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    hidden_units = [30, 10],
    # the model must choose between 3 classes 
    n_classes = 3 
)


#TRAINING 

classifier.train(
    input_fn = lambda: input_fn(train, train_y, training = True), 
    steps = 5000
)# we include a lambda to avoid creating an inner function previously










Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
# evaluation 


eval_result = classifier.evaluate(
    input_fn = lambda: input_fn(test, test_y, training = False)
)
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.867



In [7]:
# Prediction 


def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))


Please type numeric values as prompted.
SepalLength: 3.34
SepalWidth: 5.2
PetalLength: 23
PetalLength: 5.3
PetalWidth: 34.4
Prediction is "Virginica" (100.0%)
